In [2]:
import torch
torch.cuda.empty_cache()

In [3]:
from tqdm.notebook import tqdm

In [4]:
#file variables

#pretrained_bert = 'allenai/scibert_scivocab_uncased'
#pretrained_bert = 'dmis-lab/biobert-base-cased-v1.2'
#pretrained_bert = 'cimm-kzn/endr-bert'
#pretrained_bert = 'SpanBERT/spanbert-base-cased'
#pretrained_bert = 'emilyalsentzer/Bio_ClinicalBERT'
#pretrained_bert = 'dmis-lab/biobert-v1.1'
#pretrained_bert = 'allenai/biomed_roberta_base' 
#pretrained_bert = 'bert-large-uncased'
#pretrained_bert = 'microsoft/BiomedNLP-BiomedBERT-large-uncased-abstract'
pretrained_bert = 'michiyasunaga/BioLinkBERT-base'
#pretrained_bert = 'microsoft/BiomedNLP-BiomedBERT-large-uncased-abstract'
#pretrained_bert = 'microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext'
#pretrained_bert = 'bert-base-uncased'
#pretrained_bert = 'monologg/biobert_v1.1 pubmed'

#TRAINING FILE
use_huggingface = True
training_file = "**training**.csv"
# training_file = "**training**.csv"
column1_AEtext = "text" #"Example"
column2_AEflag = "label" #"AE Flag"
train_test_split_percent = 0.2
balance_multiplier = 1.0
#batch_size = 3
balance_me = True
#balance_me = False

#TEST FILE
test_file = "**test**.csv"
# test_file = "**test**.csv"
test_column1_AEtext = "Example"
test_column2_AEflag = "AE Flag"

In [5]:
import os 
#os.environ['WANDB_NOTEBOOK_NAME'] = 'BERTcomparison.ipynb'

In [6]:
import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jon-l-collins. Use `wandb login --relogin` to force relogin


True

In [7]:
sweep_config = {
    'method': 'random'
    }

In [8]:
metric = {
        'name': 'val_loss',
        'goal': 'minimize'
}
sweep_config['metric'] = metric

In [9]:
import datetime
print(str(datetime.datetime.now()))

2024-02-29 22:43:10.104756


In [10]:
parameters_dict = {
    'learning_rate': {
        'values': [5e-6,5e-5]
    },
    'batch_size': {
        'values': [8,16]
        },
    
    'dropout': {
          'values': [0.15,0.2,0.25]
        },
    
    'epochs': {
        'values': [10]},
    
    'weight_decay': {
        'values': [0.01,0.001]
    }   
 
}
sweep_config['parameters'] = parameters_dict
pretrained_bert2 = pretrained_bert.replace('/','')
sweep_id = wandb.sweep(sweep_config, project=f"{pretrained_bert2}_{str(datetime.datetime.now())[:10]}")

Create sweep with ID: h06m7k9y
Sweep URL: https://wandb.ai/jon-l-collins/michiyasunagaBioLinkBERT-base_2024-02-29/sweeps/h06m7k9y


In [11]:
#

In [12]:
import torch
#from tqdm.notebook import tqdm
#from tqdm import tqdm
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, BertConfig, AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import get_linear_schedule_with_warmup
import numpy as np
import time
import datetime
import random
import os
import pandas as pd

In [13]:
from datasets import load_dataset
use_huggingface = True
if use_huggingface == True:
    dataset = load_dataset("ade_corpus_v2","Ade_corpus_v2_classification")
else:
    dataset = pd.read_csv(training_file, encoding='unicode_escape')

In [14]:
if use_huggingface == True:
    #convert to pandas dataframe
    df = pd.DataFrame(dataset['train'])

In [15]:
df[column2_AEflag].value_counts()

label
0    16695
1     6821
Name: count, dtype: int64

In [16]:
num_AEs = df[column2_AEflag].value_counts()[1]
balance_number = int(num_AEs * balance_multiplier)

# num_AEs = 3000
# balance_number = int(num_AEs * balance_multiplier)

In [17]:
print(f"num_AEs = {num_AEs}\nbalance_number = {balance_number}\nbalance_multiplier = {balance_multiplier}")

num_AEs = 6821
balance_number = 6821
balance_multiplier = 1.0


In [18]:
if(balance_me == True):
    d_NoAE = df[df[column2_AEflag]==0]
    shuffle_d_NoAE = d_NoAE.sample(frac=1,random_state=42)[:balance_number]
    d_AE = df[df[column2_AEflag]==1]
    #d_AE = df[df[column2_AEflag]==1][:3000]
    df = pd.concat([shuffle_d_NoAE,d_AE])

In [19]:
df[column2_AEflag].value_counts()

label
0    6821
1    6821
Name: count, dtype: int64

In [20]:
X = df[column1_AEtext]
y = df[column2_AEflag]

In [21]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train, y_test = train_test_split(X,y, test_size=train_test_split_percent,stratify=y, random_state=42)

In [22]:
X_test.shape , y_test.shape

((2729,), (2729,))

In [23]:
X_val,X_test_data,y_val, y_test_data = train_test_split(X_test,y_test, test_size=0.1,stratify=y_test, random_state=42)

In [24]:
X_train.shape,X_test_data.shape, y_train.shape,y_test_data.shape, X_val.shape,y_val.shape

((10913,), (273,), (10913,), (273,), (2456,), (2456,))

In [25]:
#tokenizer = BertTokenizer.from_pretrained(pretrained_bert, do_lower_case=True)
#tokenizer = AutoTokenizer.from_pretrained(pretrained_bert, do_lower_case=True)
tokenizer = AutoTokenizer.from_pretrained(pretrained_bert, add_prefix_space=True, use_fast=False)

tokenizer_config.json:   0%|          | 0.00/379 [00:00<?, ?B/s]

c:\Users\jonlc\anaconda3\envs\bestpytorch\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\jonlc\.cache\huggingface\hub\models--michiyasunaga--BioLinkBERT-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/225k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [26]:
X_train.values

array(['We present a fatal case of subacute methanol toxicity with associated diffuse brain involvement, including bilateral putaminal necrosis and cerebral edema with ventricular compression.',
       'The evaluation, causes, and differential diagnosis of alopecia in a black woman are discussed.',
       'The development of safer and more effective means for VT control is progressing and is needed.',
       ...,
       'We report the case of a patient who developed polyserositis (pericardial effusion, pleural effusion, and pericarditis) after being started on clozapine, and whose symptoms remitted upon discontinuation of clozapine.',
       'This is a case of pseudoephedrine-induced intracerebral hemorrhage in a patient with an underlying vascular malformation.',
       'METHOD: Report of a case.'], dtype=object)

In [27]:
encoded_data_train = tokenizer.batch_encode_plus(X_train.values,
                                                    add_special_tokens=True,
                                                    return_attention_mask=True,
                                                    pad_to_max_length=True,
                                                    max_length=512,
                                                    return_tensors='pt')

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
c:\Users\jonlc\anaconda3\envs\bestpytorch\Lib\site-packages\transformers\tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [28]:
encoded_data_val = tokenizer.batch_encode_plus(X_val.values,
                                                    add_special_tokens=True,
                                                    return_attention_mask=True,
                                                    pad_to_max_length=True,
                                                    max_length=512,
                                                    return_tensors='pt')

In [29]:
encoded_data_test = tokenizer.batch_encode_plus(X_test_data.values,
                                                    add_special_tokens=True,
                                                    return_attention_mask=True,
                                                    pad_to_max_length=True,
                                                    max_length=512,
                                                    return_tensors='pt')

In [30]:
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
#labels = torch.tensor(dataset['AE Flag'].replace(label_dict).values)
labels_train = torch.tensor(y_train.values)

In [31]:
input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
#labels = torch.tensor(dataset['AE Flag'].replace(label_dict).values)
labels_val = torch.tensor(y_val.values)

In [32]:
input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']
#labels = torch.tensor(dataset['AE Flag'].replace(label_dict).values)
labels_test = torch.tensor(y_test_data.values)

In [33]:
from torch.utils.data import TensorDataset, random_split
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)
dataset_test = TensorDataset(input_ids_test, attention_masks_test, labels_test)

In [34]:
import matplotlib.pyplot as plt
import random
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [35]:
def ret_dataloader():
    batch_size = wandb.config.batch_size
    print(f"batch_size = {batch_size}")
    dataloader_train = DataLoader(dataset_train, sampler=RandomSampler(dataset_train), batch_size=batch_size)
    dataloader_validation = DataLoader(dataset_val, sampler = SequentialSampler(dataset_val), batch_size=batch_size)
    #dataloader_test = DataLoader(dataset_test, sampler = SequentialSampler(dataset_test), batch_size=batch_size)
    return dataloader_train, dataloader_validation

In [37]:
def ret_model():
    
    configuration = AutoConfig.from_pretrained(pretrained_bert)
    configuration.hidden_dropout_prob = wandb.config.dropout
    configuration.attention_probs_dropout_prob = wandb.config.dropout
    configuration.num_labels = 2
    configuration.output_attentions = False
    configuration.output_hidden_states = False

    model = AutoModelForSequenceClassification.from_pretrained(pretrained_bert,config=configuration)
    return model

In [38]:
from transformers import AdamW, get_linear_schedule_with_warmup

def ret_optim(model):
    print(f"learning_rate = {wandb.config.learning_rate}")
    optimizer = AdamW(model.parameters(),
                      lr = wandb.config.learning_rate, 
                      eps = 1e-8,weight_decay = wandb.config.weight_decay)
    return optimizer

In [39]:
def ret_scheduler(optimizer, dataloader_train):
    epochs = wandb.config.epochs
    total_steps = len(dataloader_train) * epochs
    scheduler = get_linear_schedule_with_warmup(optimizer, 
                                                num_warmup_steps = 0, 
                                                num_training_steps = total_steps)
    return scheduler

In [40]:
def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [41]:
import random
import numpy as np

In [42]:
from sklearn.metrics import f1_score

def f1_score_func(preds,labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

In [43]:
import random
def train():
    wandb.init()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model=ret_model()
    model.to(device)
    dataloader_train, dataloader_validation = ret_dataloader()
    optimizer = ret_optim(model)
    scheduler = ret_scheduler(optimizer, dataloader_train)
    training_stats = []
    total_t0 = time.time()
    epochs = wandb.config.epochs
    for epoch_i in range(0, epochs):
        print("")
        print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
        print('Training...')
        t0 = time.time()
        total_train_loss = 0
        model.train()
        for step, batch in enumerate(dataloader_train):
            if step % 200 == 0 and not step == 0:
                elapsed = format_time(time.time() - t0)
                print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(dataloader_train), elapsed))
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)
            model.zero_grad()
            outputs = model(b_input_ids,
                            token_type_ids=None,
                            attention_mask=b_input_mask,
                            labels=b_labels)
            loss = outputs[0]
            wandb.log({"train_batch_loss":loss.item()})
            total_train_loss += loss.item()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
        avg_train_loss = total_train_loss / len(dataloader_train)
        training_time = format_time(time.time() - t0)
        wandb.log({"train_loss": avg_train_loss, "epoch": epoch_i})
        print("")
        print("  Average training loss: {0:.2f}".format(avg_train_loss))
        print("  Training epoch took: {:}".format(training_time))
        print("")
        print("Running Validation...")
        t0 = time.time()
        model.eval()
        total_eval_loss = 0
        predictions, true_vals = [], []
        #nb_eval_steps = 0
        for batch in dataloader_validation:
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)
            with torch.no_grad():
                outputs = model(b_input_ids,
                                token_type_ids=None,
                                attention_mask=b_input_mask,
                                labels=b_labels)
                loss=outputs[0] #checkleul from here downwards
                logits=outputs[1] 
            total_eval_loss += loss.item()
            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()
            predictions.append(logits)
            true_vals.append(label_ids)

        predictions = np.concatenate(predictions, axis=0) #added
        true_vals = np.concatenate(true_vals, axis=0) #added
        val_f1 = f1_score_func(predictions, true_vals) #added
        wandb.log({"val_f1": val_f1, "epoch": epoch_i}) #added
        print("  Average validation f1: {0:.2f}".format(val_f1))
        avg_val_loss = total_eval_loss / len(dataloader_validation)
        wandb.log({"val_loss": avg_val_loss,"epoch": epoch_i})
        print("  Average validation loss: {0:.2f}".format(avg_val_loss))

In [44]:
wandb.agent(sweep_id, train)

wandb: Agent Starting Run: j8xw6ljy with config:
wandb: 	batch_size: 8
wandb: 	dropout: 0.2
wandb: 	epochs: 10
wandb: 	learning_rate: 5e-06
wandb: 	weight_decay: 0.001
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


config.json:   0%|          | 0.00/559 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at michiyasunaga/BioLinkBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


batch_size = 8
learning_rate = 5e-06

======== Epoch 1 / 10 ========
Training...


...

batch_size = 16
learning_rate = 5e-06

======== Epoch 1 / 10 ========
Training...


c:\Users\jonlc\anaconda3\envs\bestpytorch\Lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  Batch   200  of    683.    Elapsed: 0:01:24.
  Batch   400  of    683.    Elapsed: 0:02:45.
  Batch   600  of    683.    Elapsed: 0:04:06.

  Average training loss: 0.32
  Training epoch took: 0:04:41

Running Validation...
  Average validation f1: 0.93
  Average validation loss: 0.20

======== Epoch 2 / 10 ========
Training...
  Batch   200  of    683.    Elapsed: 0:01:21.
  Batch   400  of    683.    Elapsed: 0:02:41.
  Batch   600  of    683.    Elapsed: 0:04:02.

  Average training loss: 0.17
  Training epoch took: 0:04:35

Running Validation...
  Average validation f1: 0.95
  Average validation loss: 0.19

======== Epoch 3 / 10 ========
Training...
  Batch   200  of    683.    Elapsed: 0:01:21.
  Batch   400  of    683.    Elapsed: 0:02:43.
  Batch   600  of    683.    Elapsed: 0:04:04.

  Average training loss: 0.14
  Training epoch took: 0:04:38

Running Validation...
  Average validation f1: 0.95
  Average validation loss: 0.21

======== Epoch 4 / 10 ========
Training...
  Ba

epoch,▁▁▁▂▂▂▃▃▃▃▃▃▄▄▄▅▅▅▆▆▆▆▆▆▇▇▇███
train_batch_loss,██▆▁▄▅█▅▂▁▄▁▃▁▁▄▁▄▁▄▅▁▅▁▄▁▁▁▄▂▄▁▁▁▁▁▁▁▁▁
train_loss,█▄▃▂▂▂▁▁▁▁
val_f1,▁▆▆▅▇▇▇▇██
val_loss,▂▁▄▄▄▄▆█▆▆
epoch,9
train_batch_loss,0.00073
train_loss,0.07295
val_f1,0.95436
val_loss,0.23467


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
